In [62]:
from PIL import Image, ImageDraw
import math
import json
from abc import ABC, abstractmethod
from re import findall
from itertools import chain

In [78]:
class ImageManager():
    
    '''Represents an area on which all shapes from an input file are drawn'''
    
    def __init__(self, width, height, bg_color, fg_color=None):
            
        self.width = width
        self.height = height
        self.bg_color = bg_color
        self.fg_color = fg_color # dafuq is fg_color?
        
        self._image = Image.new(size=(width, height), color=bg_color, mode='RGB')
        self._surface = ImageDraw.Draw(self._image)
        
    def draw(self, shape):
        
        shape.draw(self._surface)
            
    def display(self):
        
        self._image.show()
        
    def save(self, file_name):
        
        self._image.save(file_name)
            
            

class AbstractShape(ABC):
    
    '''Represents a single shape that may be drawn on canvas'''
    
    def __init__(self, color):
        self.color = color
        
    @abstractmethod
    def draw(self, surface):
        pass
    
    
class Point(AbstractShape):
    
    '''Point shape'''
    
    def __init__(self, x, y, color):
        
        super().__init__(color)
        self.x_coord = x
        self.y_coord = y

    def draw(self, surface):
        
        surface.point((self.x_coord, self.y_coord), fill=self.color)
        
        
class Circle(AbstractShape):
    
    '''Circle shape'''
    
    def __init__(self, x, y, radius, color):
        
        super().__init__(color)
        self.x_coord = x
        self.y_coord = y
        self.radius = radius

    def draw(self, surface):
        
        surface.ellipse((self.x_coord - self.radius,
                         self.y_coord - self.radius,
                         self.x_coord + self.radius,
                         self.y_coord + self.radius),
                        fill=self.color)
        
        
class Square(AbstractShape):
    
    '''Square shape'''
    
    def __init__(self, x, y, size, color):
        
        super().__init__(color)
        self.x_coord = x
        self.y_coord = y
        self.size = size

    def draw(self, surface):
        
        surface.rectangle((self.x_coord - (self.size // 2), 
                           self.y_coord - (self.size // 2), 
                           self.x_coord + (self.size // 2), 
                           self.y_coord + (self.size // 2)),
                          fill=self.color)
        
        
class Rectangle(AbstractShape):
    
    '''Rectangle shape'''
    
    def __init__(self, x, y, height, width, color):
        
        super().__init__(color)
        self.x_coord = x
        self.y_coord = y
        self.height = height
        self.width = width

    def draw(self, surface):
        
        surface.rectangle((self.x_coord - (self.width // 2), 
                           self.y_coord - (self.height // 2), 
                           self.x_coord + (self.width // 2), 
                           self.y_coord + (self.height // 2)),
                          fill=self.color)
        
        
class Polygon(AbstractShape):
    
    '''Polygon shape'''
    
    def __init__(self, points, color):
        
        super().__init__(color)
        self.points = points
        
    def draw(self, surface):
        
        surface.polygon(list(chain(*self.points)), fill=self.color)
    
    

class FileParser():

    '''Used for parsing all the data from the input file'''
    
    _figure_constructors = {
        'point':Point,
        'circle': Circle,
        'square': Square,
        'rectangle': Rectangle,
        'polygon': Polygon
    }
    
    def __init__(self, json_file):
        
        with open(json_file, 'r') as file:
            
            self._raw_dict = json.load(file)
            
            self.color_palette = self._raw_dict.get('Palette', {})
            
            self.screen_parameters = self._raw_dict['Screen']
            self.screen_parameters['bg_color'] = self._parse_color(self.screen_parameters['bg_color'])
            self.screen_parameters['fg_color'] = self._parse_color(self.screen_parameters['fg_color'])
            
            raw_figures = self._raw_dict['Figures']
            self._parse_figures(raw_figures)
                
    def _parse_color(self, string_color):
        
        if string_color.startswith('#'):
            return string_color
        
        if string_color in self.color_palette.keys():
            return self.color_palette[string_color]
        
        (r, g, b) = findall('[0-9]+', string_color)
        return (int(r), int(g), int(b))
    

    def _parse_figures(self, raw_figures):
        
        figures = []
        
        for figure_dict in raw_figures:
            
            print(figure_dict)
            figure_type = figure_dict.pop('type')
            figure_dict['color'] = self._parse_color(figure_dict.get('color', '#000000'))
            
            constructor = FileParser._figure_constructors[figure_type]
            
            figures.append(constructor(**figure_dict))
            
        self.figures = figures
        

In [60]:
im = ImageManager(bg_color=(255, 0, 0), height=400, width=600)
cr = Circle(color=(0, 255, 0), radius=40, x=200, y=180)
pl = Polygon(color='yellow', points=[10, 10, 30, 40, 100, 20, 200, 150, 250, 350, 200, 300, 30, 390])
pt = Point(color='blue', x=400, y=350)
sq = Square(color='pink', x=500, y=300, size=28)
rc = Rectangle(color='purple', x=300, y=200, width=300, height=200)
im.draw(rc)
im.draw(cr)
im.draw(pl)
im.draw(pt)
im.draw(sq)
im.display()

[10, 10, 30, 40, 100, 20, 200, 150, 250, 350, 200, 300, 30, 390] ::: 'yellow'


In [80]:
fp = FileParser('sample.json')
im = ImageManager(**fp.screen_parameters)
for figure in fp.figures:
    im.draw(figure)
    
im.display()

{'type': 'circle', 'x': 500, 'y': 500, 'color': 'secondary', 'radius': 150}
{'type': 'polygon', 'color': 'primary', 'points': [[0, -100], [20, -20], [100, 0], [20, 20], [0, 100], [-20, 20], [-100, 0], [-20, -20]]}
{'type': 'point', 'x': 500, 'y': 500, 'color': 'secondary'}
{'size': 100, 'type': 'square', 'x': 100, 'y': 100, 'color': 'primary'}
{'size': 100, 'type': 'square', 'x': 100, 'y': 800, 'color': 'primary'}
{'size': 100, 'type': 'square', 'x': 800, 'y': 100, 'color': 'primary'}
{'size': 100, 'type': 'square', 'x': 800, 'y': 800, 'color': 'primary'}
{'width': 400, 'type': 'rectangle', 'height': 100, 'y': 100, 'color': 'secondary', 'x': 300}
{'width': 100, 'type': 'rectangle', 'height': 400, 'y': 300, 'color': 'secondary', 'x': 100}
{'width': 400, 'type': 'rectangle', 'height': 100, 'y': 800, 'color': 'secondary', 'x': 300}
{'width': 100, 'type': 'rectangle', 'height': 400, 'y': 300, 'color': 'secondary', 'x': 800}
